In [4]:
import os
import cv2
from ultralytics import YOLO

# ---------------------------
# CONFIG
# ---------------------------
MODEL_PATH = "models/fish_yolov8/weights/best.pt"
TEST_IMAGES = "data/aquarium_pretrain/test/images"
TEST_LABELS = "data/aquarium_pretrain/test/labels"

SAVE_MISCLASSIFIED = True

OUT_PRED_DIR = "misclassified_yolo"              # predictions (RED)
OUT_GT_DIR = "misclassified_ground_truth"        # actual labels (GREEN)

CLASSES = ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']

os.makedirs(OUT_PRED_DIR, exist_ok=True)
os.makedirs(OUT_GT_DIR, exist_ok=True)

model = YOLO(MODEL_PATH)

# ---------------------------
# HELPER: Read YOLO GT labels
# ---------------------------
def read_yolo_label(label_path, img_w, img_h):
    boxes = []
    if not os.path.exists(label_path):
        return boxes

    with open(label_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls = int(parts[0])
            xc, yc, w, h = map(float, parts[1:])

            # Convert normalized → pixel xyxy
            x1 = int((xc - w / 2) * img_w)
            y1 = int((yc - h / 2) * img_h)
            x2 = int((xc + w / 2) * img_w)
            y2 = int((yc + h / 2) * img_h)

            boxes.append((cls, x1, y1, x2, y2))

    return boxes


# ---------------------------
# MAIN LOOP
# ---------------------------
misclassified = []

for img_name in os.listdir(TEST_IMAGES):
    if not img_name.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    img_path = os.path.join(TEST_IMAGES, img_name)
    label_path = os.path.join(
        TEST_LABELS, img_name.replace(".jpg", ".txt").replace(".png", ".txt")
    )

    img = cv2.imread(img_path)
    img_h, img_w = img.shape[:2]

    # --- Read ground truth ---
    gt_boxes = read_yolo_label(label_path, img_w, img_h)
    gt_classes = [b[0] for b in gt_boxes]

    # --- Predict ---
    results = model.predict(img_path, conf=0.25, verbose=False)[0]
    pred_classes = [
        int(box.cls[0]) for box in results.boxes
    ] if results.boxes is not None else []

    # ---------------------------
    # If misclassified
    # ---------------------------
    if sorted(gt_classes) != sorted(pred_classes):
        misclassified.append({
            "image": img_name,
            "gt": [CLASSES[c] for c in gt_classes],
            "pred": [CLASSES[c] for c in pred_classes]
        })

        if SAVE_MISCLASSIFIED:

            # COPY of original for prediction drawing
            pred_img = img.copy()

            # COPY of original for GT drawing
            gt_img = img.copy()

            # ---------------------------
            # DRAW GROUND TRUTH (GREEN)
            # ---------------------------
            for cls, x1, y1, x2, y2 in gt_boxes:
                cls_name = CLASSES[cls]

                cv2.rectangle(gt_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(
                    gt_img,
                    f"GT: {cls_name}",
                    (x1, max(y1 - 8, 5)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (0, 255, 0),
                    2
                )

            # Save ground-truth only image
            cv2.imwrite(os.path.join(OUT_GT_DIR, img_name), gt_img)

            # ---------------------------
            # DRAW PREDICTIONS (RED)
            # ---------------------------
            if results.boxes is not None:
                for box in results.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cls_id = int(box.cls[0])
                    cls_name = CLASSES[cls_id]

                    cv2.rectangle(pred_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.putText(
                        pred_img,
                        f"Pred: {cls_name}",
                        (x1, max(y1 - 8, 5)),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,
                        (0, 0, 255),
                        2
                    )

            # Save predicted result only
            cv2.imwrite(os.path.join(OUT_PRED_DIR, img_name), pred_img)



# ---------------------------
# SUMMARY
# ---------------------------
print("\nTotal Misclassified:", len(misclassified))
for m in misclassified:
    print(m)



Total Misclassified: 49
{'image': 'IMG_2446_jpeg_jpg.rf.06ee05e92df8e3c33073147d8f595211.jpg', 'gt': ['shark', 'fish', 'fish', 'fish', 'fish', 'fish', 'fish', 'shark'], 'pred': ['fish', 'fish', 'fish', 'shark', 'fish', 'fish', 'fish', 'fish', 'shark', 'shark']}
{'image': 'IMG_2395_jpeg_jpg.rf.9f1503ad3b7a7c7938daed057cc4e9bc.jpg', 'gt': ['fish', 'fish', 'fish', 'fish', 'fish', 'fish', 'fish', 'fish'], 'pred': ['fish', 'fish', 'fish', 'fish', 'fish']}
{'image': 'IMG_2380_jpeg_jpg.rf.a23809682eb1466c1136ca0f55de8fb5.jpg', 'gt': ['fish'], 'pred': ['fish', 'fish', 'fish', 'fish']}
{'image': 'IMG_2347_jpeg_jpg.rf.7c71ac4b9301eb358cd4a832844dedcb.jpg', 'gt': ['penguin', 'penguin'], 'pred': ['penguin']}
{'image': 'IMG_2379_jpeg_jpg.rf.7dc3160c937072d26d4624c6c48e904d.jpg', 'gt': ['fish'], 'pred': ['fish', 'shark']}
{'image': 'IMG_8582_MOV-5_jpg.rf.9d7a26fbf145ce39ab0831b4e6bc1f1e.jpg', 'gt': ['stingray', 'fish', 'fish', 'fish', 'fish', 'fish', 'fish'], 'pred': ['fish', 'fish']}
{'image': 'IM

In [5]:
from ultralytics import YOLO

model = YOLO("models/fish_yolov84/weights/best.pt")

# Run evaluation on your test set
results = model.val(data="data/dataset.yaml")   # or your dataset YAML

# Print metrics
print("mAP50:", results.box.map50)
print("mAP50-95:", results.box.map)
print("Precision:", results.box.mp)
print("Recall:", results.box.mr)

# Class-wise AP
print("\nClass-wise AP:")
print(results.box.maps)


Ultralytics 8.3.228 🚀 Python-3.11.2 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
Model summary (fused): 72 layers, 11,128,293 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5924.0±1418.3 MB/s, size: 369.4 KB)
val: Scanning /workspace/Jaasia/Fish_Detection/data/labels/valid_aug.cache... 441 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 441/441 1.9Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 28/28 16.3it/s 1.7s0.1s
                   all        441       2660      0.818      0.679      0.759      0.469
                  fish        175        903      0.889      0.606       0.76       0.44
             jellyfish         42        679      0.912      0.829      0.918        0.6
               penguin         70        385      0.573      0.623      0.587       0.28
                puffin         63        315      0.712      0.518      0.618       0.32
